## 1. 主成分分析
> 与えられた特徴量から新たな特徴量`主成分`を作り出し、元の特徴量よりも少ない数の変数（次元）でデータを説明する手法<br>

![alt text](image.png)

- 中心化

- 共分散行列　Covariance Matrix
> 多次元であっても、共分散（二つの変数の関係）と分散を行列に並べているだけ
$$
\Sigma = 
\begin{pmatrix}
\text{Var}(x) & \text{Cov}(x, y) & \text{Cov}(x, z) \\
\text{Cov}(y, x) & \text{Var}(y) & \text{Cov}(y, z) \\
\text{Cov}(z, x) & \text{Cov}(z, y) & \text{Var}(z)
\end{pmatrix}
$$

In [ ]:
import tensorflow as tf

class PCATensorFlow:
    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None
        self.mean_ = None
        self.eigenvalues_ = None

    def fit(self, X):
        """
        Args:
            X (tf.Tensor): Input data. Shape [n_samples, n_features]

        中心化
           まず各特徴量の平均を計算し、データから引く
        """
        # tf.cast で型を float32 に統一（GPU演算の基本）
        X = tf.cast(X, tf.float32)
        
        # 1. 中心化
        self.mean_ = tf.reduce_mean(X, axis=0)
        X_centered = X - self.mean_
        
        n_samples = tf.cast(tf.shape(X)[0], tf.float32)

        # 2. 共分散行列の計算
        # NumPyの dot ではなく、TensorFlowの行列積関数を使います。
        # ヒント: [n_features, n_samples] x [n_samples, n_features] の形にする必要があります。
        # transpose_a=True (Xを転置) または transpose_b=True (後ろを転置) の引数を活用します。
        cov_matrix = [Q1] / (n_samples - 1)

        # 3. 固有値分解 (Self-Adjoint Eigendecomposition)
        # 対称行列専用の高速ソルバー tf.linalg.eigh を使います。
        # これも昇順（小さい順）で返ってきます。
        eigenvalues, eigenvectors = tf.linalg.eigh(cov_matrix)

        # 4. ソート（降順）
        # TensorFlowにはスライシングの他に、逆順にする専用関数があります（スライスでも可）。
        # ここではインデックス順序を反転させる処理を記述してください。
        # 例: Pythonのスライス記法 [::-1] はTensorでも有効です。
        sorted_indices = tf.argsort(eigenvalues, direction='DESCENDING') 
        # ↑ TFでは便利な引数がありますが、もしスライスで書くなら？
        # 今回はシンプルに [Q2] (変数名とスライス) を答えてください。
        # ※ tf.argsortを使わず、すでに取得した eigenvalues, eigenvectors をどう逆順にするか
        
        # 修正: わかりやすくするために、tf.argsortを使わずに
        # 「得られた結果を逆順にするスライス操作」を答えてください。
        self.eigenvalues_ = eigenvalues[[Q2]]
        self.eigenvectors_ = eigenvectors[:, [Q2]]

        # 上位n成分を取得
        self.components_ = self.eigenvectors_[:, :self.n_components]

    def transform(self, X):
        X = tf.cast(X, tf.float32)
        X_centered = X - self.mean_
        
        # 5. 射影
        # TensorFlowの行列積で行ってください。
        return [Q3]

### 2. 主成分分析